In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import networkx as nx
import osmnx
from datetime import datetime,timedelta
from fastprogress.fastprogress import master_bar, progress_bar
import os
pd.set_option('mode.chained_assignment','raise')

/home/lulu/.pyenv/versions/3.9.7/envs/study_1_venv/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.9.1-CAPI-1.14.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [2]:
def extract_streets_from_OSM(city_boundary, destination_path):
    '''
    This function extracts the street network in a polygon and saves it.
    
           Parameters:
                    city_boundary (gpd.GeoDataFrame): polygon containing the city boundaries.
                    destination_path (str): Path where the street network should be stored.

            Returns:
                    None
    '''
    # Taking the convex hull of the city for the polygon requested to the API.
    polygon = city_boundary.convex_hull
    polygon = polygon.to_crs('EPSG:28992').buffer(200)
    # CRS 4326 is needed to use osmnx.
    polygon = polygon.to_crs('EPSG:4326')
    net = osmnx.graph_from_polygon(polygon.values[0], 
                                   network_type = 'all', 
                                   simplify = False)
    
    net = osmnx.utils_graph.get_undirected(net)
    
    osmnx.io.save_graph_geopackage(net, 
                                   filepath = destination_path)

In [6]:
directory_street_layout = '../data/raw_data/street_data/'
directory_city_boundary = '../data/processed_data/city_boundary/'

list_files = os.listdir(directory_city_boundary)
# Removing README file.
list_files.remove('city_boundary_README.mkd')
pg = progress_bar(range(len(list_files)))
for i in pg:
    file = list_files[i]
    # Removing the extension.
    city_name = file[:-5]
    pg.comment = city_name
    if not os.path.isfile(directory_street_layout + city_name + '_streets.gpkg'):
        city_boundary = gpd.read_file(directory_city_boundary + file)
        extract_streets_from_OSM(city_boundary,
                                 directory_street_layout + city_name + '_streets.gpkg')